In [1]:
import pandas as pd
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import os
print(os.getcwd())

/home/amaan-ali/Desktop/coding/python_projects/data_analysis/data_analysis/bin/code


In [ ]:
movies = pd.read_csv('data/movies-final.csv')

In [6]:
ps = PorterStemmer()

In [18]:
movies.head(10)

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
5,293660,Deadpool,7.606,28894,Released,2016-02-09,783100000,108,False,/en971MEXui9diirXlogOrPKmsEn.jpg,...,Deadpool,The origin story of former Special Forces oper...,72.735,/zq8Cl3PNIDGU3iWNRoc5nEZ6pCe.jpg,Witness the beginning of a happy ending.,"Action, Adventure, Comedy","20th Century Fox, The Donners' Company, Genre ...",United States of America,English,"superhero, anti hero, mercenary, based on comi..."
6,299536,Avengers: Infinity War,8.255,27713,Released,2018-04-25,2052415039,149,False,/mDfJG3LC3Dqb67AZ52x3Z0jU0uB.jpg,...,Avengers: Infinity War,As the Avengers and their allies have continue...,154.340,/7WsyChQLEftFiDOVTGkv3hFpyyt.jpg,An entire universe. Once and for all.,"Adventure, Action, Science Fiction",Marvel Studios,United States of America,"English, Xhosa","sacrifice, magic, superhero, based on comic, s..."
7,550,Fight Club,8.438,27238,Released,1999-10-15,100853753,139,False,/hZkgoQYus5vegHoetLkCJzb17zJ.jpg,...,Fight Club,A ticking-time-bomb insomniac and a slippery s...,69.498,/pB8BM7pdSp6B6Ih7QZ4DrQ3PmJK.jpg,Mischief. Mayhem. Soap.,Drama,"Regency Enterprises, Fox 2000 Pictures, Taurus...",United States of America,English,"dual identity, rage and hate, based on novel o..."
8,118340,Guardians of the Galaxy,7.906,26638,Released,2014-07-30,772776600,121,False,/uLtVbjvS1O7gXL8lUOwsFOH4man.jpg,...,Guardians of the Galaxy,"Light years from Earth, 26 years after being a...",33.255,/r7vmZjiyZw9rpJMQJdXpjgiCOk9.jpg,All heroes start somewhere.,"Action, Science Fiction, Adventure",Marvel Studios,United States of America,English,"spacec

In [7]:
def process_text(text):
    if not isinstance(text ,str): # handle NaN or missing value
        return ""
    result = []
    for i in text.split():
        result.append(ps.stem(i))

    return " ".join(result)

In [8]:
movies_subset = movies[['id', 'title', 'overview']].copy()

In [9]:
movies_subset.head(5)

,id,title,overview
0,27205,Inception,"Cobb, a skilled thief who commits corporate es..."
1,157336,Interstellar,The adventures of a group of explorers who mak...
2,155,The Dark Knight,Batman raises the stakes in his war on crime. ...
3,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
4,24428,The Avengers,When an unexpected enemy emerges and threatens...


In [10]:
sample_movies = movies_subset.head(5000)

In [11]:
sample_movies['overview'] = sample_movies['overview'].apply(process_text)

/tmp/ipykernel_123461/875229042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_movies['overview'] = sample_movies['overview'].apply(process_text)


In [12]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [13]:
vectors = cv.fit_transform(sample_movies['overview'])

In [14]:
similarity = cosine_similarity(vectors)

In [ ]:
# def recommend_movie(movie):
    
#     if movie not in sample_movies['title'].values:
#         print('movie does not exists')
#         return
#     movie_index = sample_movies[sample_movies['title'] == movie].index[0]
#     distance = similarity[movie_index]
#     similar_movies_list = sorted(list(enumerate(distance)), reverse=True, key= lambda x:x[1])[1:6]

#     for i in similar_movies_list:
#         print(sample_movies.iloc[i[0]].title)
def recommend_movie(movie_id : int):
    
    if movie_id not in sample_movies['id'].values:
        print(f"movie with id ${movie_id} does not exists.")
        return
    movie_index = sample_movies[sample_movies['id'] == movie_id].index[0]
    distance = similarity[movie_index]
    similar_movies_list = sorted(list(enumerate(distance)), reverse=True, key= lambda x:x[1])[1:6]

    for i in similar_movies_list:
        print(sample_movies.iloc[i[0]].title)

In [25]:
sample_movies.head()

,id,title,overview
0,27205,Inception,"cobb, a skill thief who commit corpor espionag..."
1,157336,Interstellar,the adventur of a group of explor who make use...
2,155,The Dark Knight,batman rais the stake in hi war on crime. with...
3,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
4,24428,The Avengers,when an unexpect enemi emerg and threaten glob...


In [27]:
recommend_movie(19995)

Tears of the Sun
Apollo 18
First Man
Venom
Battle: Los Angeles
